In [1]:
import numpy as np
import gym
import time
from IPython.display import clear_output
env = gym.make("Taxi-v3") #Gọi môi trường ra

In [2]:
env.env.s=env.encode(1,1,2,0)
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Naive Learning

In [3]:
state_size = env.observation_space.n
action_size = env.action_space.n

q_table = np.zeros((state_size, action_size))

FILE_SAVE_Naive = "q_table_naive.npy"
total_episodes = 500      # Total episodes
total_test_episodes = 10     # Total test episodes
max_steps = 99                # Max steps per episode
           # Exponential decay rate for exploration prob

In [4]:
import time
for episode in range(total_episodes):
    state = env.reset()   # reset lại môi trường
    done = False          # đã hoàn thành trả khách hay chưa
    for step in range(max_steps):
        clear_output(wait=True)
        print("episode: ",episode)
        if  np.max(q_table[state]) ==0:         # nếu trong state q_value đều bằng 0 thì chọn đại 1 action
            action=np.random.randint(0,action_size)
        else:
            action = np.argmax(q_table[state])  # chọn action có q_value lớn nhất

        new_state, reward, done, _ = env.step(action)   # thực hiện hành động để nhận reward và state, action mới

        q_table[state,action] += reward # cập nhật q_table
        state = new_state
        if done:       # nếu taxi có hành vi trả khách thì kết thúc episode
            break

episode:  499


In [5]:
# lưu lại q_table
np.save(FILE_SAVE_Naive,q_table)

In [6]:
# từ q_table bắt đầu chơi thử
q_table=np.load("q_table_naive.npy")
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    

    for step in range(max_steps):
        print("****************************************************")
        print("EPISODE ", episode)
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, info = env.step(action)
        env.render() # hiển thị trò chơi
        total_rewards += reward
        print(total_rewards)
        time.sleep(0.5)
        clear_output(wait=True)
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

****************************************************
EPISODE  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
-20


KeyboardInterrupt: ignored

Q-Learning

In [7]:
env = gym.make("Taxi-v3")
state_size = env.observation_space.n
action_size = env.action_space.n

q_table = np.zeros((state_size, action_size))

FILE_SAVE_Qlearning = "q_table_qlearning.npy"
total_episodes = 5000         # Total episodes
total_test_episodes = 10    # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
discount_rate = 0.95         # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [8]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [9]:
import time
for episode in range(total_episodes):
    state = env.reset()
    done = False
    clear_output(wait=True)
    print("episode: ",episode)
    for step in range(max_steps):
        #clear_output(wait=True)
        
        epsilon = min(min_epsilon, epsilon*decay_rate)
        # kiểm tra xem agent dùng exploi hay explor
        if np.random.rand() < epsilon:
            # exploration
            action = np.random.randint(0, action_size)
        else:
            # exploitation
            if np.max(q_table[state])==0:
                action=np.random.randint(0,action_size)
            else:
                action = np.argmax(q_table[state])
            
        # nhận reward và state tiếp theo
        new_state, reward, done, _ = env.step(action)

        # cập nhật q_table theo Bellman equation
        update = reward + discount_rate*q_table[new_state].max() - q_table[state,action]
        q_table[state,action] = q_table[state,action] + learning_rate*update
        state = new_state
        if done:
            break

episode:  4999


In [10]:
# lưu lại q_table
np.save(FILE_SAVE_Qlearning,q_table)

In [11]:
# từ q_table bắt đầu chơi thử
#q_table=np.load("q_table_qlearning.npy")
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    

    for step in range(max_steps):
        print("****************************************************")
        print("EPISODE ", episode)
        
        action = np.argmax(q_table[state,:])
        new_state, reward, done, info = env.step(action)
        env.render() # hiển thị trò chơi
        total_rewards += reward
        print(total_rewards)
        time.sleep(0.5)
        clear_output(wait=True)
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
        
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 8.2


1. Giải thích vì sao khi dùng Naive-Learning thì xe taxi chỉ đứng yên một chỗ?
##### Naive-learning luôn chọn hành động có lợi nhất dựa trên những gì quan sát được. Nếu giá trị bằng 0 cho tất cả những hành động thì sẽ chọn một hành động ngẫu nhiên. Sau mỗi bước đi nếu như không đón khách hay trả khách được thì taxi sẽ bị -1 reward nên taxi sẽ chọn hành có lợi nhất là đứng yên một chỗ.
2. Giải thích vì sao khi dùng Q-Leaning thì xe taxi có thể đón và trả khách được?
##### Q-learning đầu tiên sẽ chọn hành động nhất dựa trên Q-table. Nhưng đôi khi thuật toán sẽ gamble và chọn một hành động ngẫu nhiên. Đó là lý do vì sao taxi có thể di chuyển đến cái điểm đón và trả khách.
3. Tìm hiểu một game khác trên OpenAI và thiết lập cho agent chơi được

## Game Frozen-Lake
#### Ta sẽ khiển một nhân vật đi trên bề mặt băng với
#### S: điểm bắt đầu, an toàn
#### F: bề mặt băng, an toàn
#### H: lỗ, rơi xuống là thua
#### G: đích đến chến thắng
#### Trò chơi kết thúc khi bạn vền đích hoặc rơi cuống lỗ. Nếu về đích sẽ nhận được 1 reward, người lại là 0 reward


In [ ]:
env = gym.make("FrozenLake-v0") #Gọi môi trường ra
#env = gym.make("Blackjack-v0")
env.render()


SFFF
FHFH
FFFH
HFFG


In [ ]:
env = gym.make("FrozenLake-v0")
state_size = env.observation_space.n
action_size = env.action_space.n

q_table = np.zeros((state_size, action_size))

FILE_SAVE_Qlearning = "q_table_qlearning.npy"
total_episodes = 50000         # Total episodes
total_test_episodes = 100    # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
discount_rate = 0.95         # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [ ]:
import time
for episode in range(total_episodes):
    state = env.reset()
    done = False
    clear_output(wait=True)
    print("episode: ",episode)
    for step in range(max_steps):
        #clear_output(wait=True)
        
        epsilon = min(min_epsilon, epsilon*decay_rate)
        # kiểm tra xem agent dùng exploi hay explor
        if np.random.rand() < epsilon:
            # exploration
            action = np.random.randint(0, action_size)
        else:
            # exploitation
            if np.max(q_table[state])==0:
                action=np.random.randint(0,action_size)
            else:
                action = np.argmax(q_table[state])
            
        # nhận reward và state tiếp theo
        new_state, reward, done, _ = env.step(action)

        # cập nhật q_table theo Bellman equation
        update = reward + discount_rate*q_table[new_state].max() - q_table[state,action]
        q_table[state,action] = q_table[state,action] + learning_rate*update
        state = new_state
        if done:
            break

episode:  49999


In [ ]:
# từ q_table bắt đầu chơi thử
#q_table=np.load("q_table_qlearning.npy")
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    

    for step in range(max_steps):
        print("****************************************************")
        print("EPISODE ", episode)
        
        action = np.argmax(q_table[state,:])
        new_state, reward, done, info = env.step(action)
        env.render() # hiển thị trò chơi
        total_rewards += reward
        print(total_rewards)
        time.sleep(0.5)
        clear_output(wait=True)
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
        
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 0.03
